In [1]:
import os
import cv2
import random
import math
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/Users/vedantpadole/Desktop/Research/finalized')

all_classes = os.listdir('/Users/vedantpadole/Desktop/Research/finalized')
print (all_classes)  
len(all_classes)

['your', 'ready', 'safe', 'order', 'fork', 'repeat', 'sandwich', 'biscuit', 'please', 'wait', 'sugar', 'cup', 'sweet', 'ac', 'refill', 'lettuce', 'bill', 'thank-you', 'water', 'no', 'vegetables', 'dessert', 'pop corn', 'would', 'bitter', 'milk', 'enjoy', 'cash', 'allergy', 'pepsi', 'coca-cola', 'is', 'spicy', 'nacho', 'cost', 'fresh', 'ice cream', 'chair', 'cheese', 'napkin', 'good night', 'mayonnaise', 'about', 'brownie', 'soda', 'bye', 'coupon', 'small', 'mustard', 'urgent', 'credit card', 'french fries', 'ketchup', 'you_ll', 'manager', 'salt', 'eat', 'total', 'receipt', 'juicy', 'cold', 'extra', 'lid', 'yogurt', 'warm', 'menu', 'ingredients', 'burger', 'alcohol', 'relish', 'eggs', 'chicken', 'hot', 'what', 'tissues', 'pickle', 'spoon', 'bread', 'additional', 'hello', 'straw', 'sauce', 'lactose', 'bag', 'drive', 'pizza', 'bacon', 'pepper', 'tomato', 'sorry', 'coke', 'drink', 'which', 'gluten free', 'I_ve', 'cake', 'barbecue']


97

In [3]:
IMAGE_HEIGHT,IMAGE_WIDTH=128,128
SEQUENCE_LENGTH=10
DATASET='/Users/vedantpadole/Desktop/Research/finalized'
CLASSES_LIST=all_classes

In [4]:
def frames_extraction(video_path):
  frames_list=[]
  video_reader=cv2.VideoCapture(video_path)
  video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)
  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
    success,frame=video_reader.read()
    if not success:
      break
    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255
    frames_list.append(normalized_frame)
  video_reader.release()
  return frames_list

In [5]:
def create_dataset():
  features=[]
  labels=[]
  video_files_path=[]
  for class_index,class_name in enumerate(CLASSES_LIST):
    print(f'Extracting the data of class:  {class_name}')
    files_list=os.listdir(os.path.join(DATASET,class_name))
    for file_name in files_list:
      video_file_path=os.path.join(DATASET,class_name,file_name)
      frames=frames_extraction(video_file_path)
      if len(frames)==SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_path.append(video_file_path)
  features=np.asarray(features)
  labels=np.array(labels)
  return features,labels,video_files_path

In [6]:
features,labels,video_files_path=create_dataset()

Extracting the data of class:  your
Extracting the data of class:  ready
Extracting the data of class:  safe
Extracting the data of class:  order
Extracting the data of class:  fork
Extracting the data of class:  repeat
Extracting the data of class:  sandwich
Extracting the data of class:  biscuit
Extracting the data of class:  please
Extracting the data of class:  wait
Extracting the data of class:  sugar
Extracting the data of class:  cup
Extracting the data of class:  sweet
Extracting the data of class:  ac
Extracting the data of class:  refill
Extracting the data of class:  lettuce
Extracting the data of class:  bill
Extracting the data of class:  thank-you
Extracting the data of class:  water
Extracting the data of class:  no
Extracting the data of class:  vegetables
Extracting the data of class:  dessert
Extracting the data of class:  pop corn
Extracting the data of class:  would
Extracting the data of class:  bitter
Extracting the data of class:  milk
Extracting the data of clas

In [7]:
from keras.utils import to_categorical
one_hot_encoded_labels=to_categorical(labels)

In [8]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [9]:
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test=train_test_split(features,one_hot_encoded_labels,test_size=0.2,shuffle=True,random_state=seed_constant)

In [20]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,TimeDistributed,Dropout,Flatten,Dense,LSTM
model=Sequential()
model.add(TimeDistributed(Conv2D(16,(3,3),padding='same',activation='relu'),input_shape=(SEQUENCE_LENGTH,IMAGE_HEIGHT,IMAGE_WIDTH,3)))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv2D(32,(3,3),padding='same',activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64))
model.add(Dense(len(CLASSES_LIST),activation='softmax'))


In [21]:
from keras.callbacks import EarlyStopping
early_stopping_call=EarlyStopping(monitor='val_loss',patience=10,mode='min',restore_best_weights=True)
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
r=model.fit(x=features_train,y=labels_train,epochs=15,batch_size=32,shuffle=True,validation_split=0.2,callbacks=[early_stopping_call])

Epoch 1/15
23/40 [================>.............] - ETA: 53s - loss: 4.5347 - accuracy: 0.0340

KeyboardInterrupt: 